### Run this first to setup django in this notebook

In [ ]:
import os, django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "backend.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

### Import django assets

In [ ]:
from weather.models import Entry

### Import other libraries

### Script starts here